In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv("ESO_export.csv")
df = df.sort_values("obs_date").reset_index(drop=True)

# generate 100 equally spaced “positions” through the 3600 rows
target_idxs = np.linspace(0, len(df)-1, 100).round().astype(int)
sample = df.iloc[target_idxs]

# now sample["ARCFILE"] or sample["URL"] gives the 100 files to retrieve


In [24]:
print(sample["access_url"])

0       http://archive.eso.org/datalink/links?ID=ivo:/...
36      http://archive.eso.org/datalink/links?ID=ivo:/...
73      http://archive.eso.org/datalink/links?ID=ivo:/...
109     http://archive.eso.org/datalink/links?ID=ivo:/...
145     http://archive.eso.org/datalink/links?ID=ivo:/...
                              ...                        
3454    http://archive.eso.org/datalink/links?ID=ivo:/...
3490    http://archive.eso.org/datalink/links?ID=ivo:/...
3526    http://archive.eso.org/datalink/links?ID=ivo:/...
3563    http://archive.eso.org/datalink/links?ID=ivo:/...
3599    http://archive.eso.org/datalink/links?ID=ivo:/...
Name: access_url, Length: 100, dtype: object


In [27]:
import pandas as pd
import numpy as np
import io
import requests
from astropy.io.votable import parse_single_table

# 1) Load & sort
df = pd.read_csv("ESO_export.csv")
df = df.sort_values("obs_date").reset_index(drop=True)

# 2) Pick 100 evenly-spaced exposures
idxs = np.linspace(0, len(df)-1, 100).round().astype(int)
sample = df.iloc[idxs].copy()

# 3) Function: turn DataLink URL into the real FITS URL
def get_fits_url(datalink_url):
    # fetch the VOTable XML
    r = requests.get(datalink_url)
    print(r)
    r.raise_for_status()
    # parse its single table
    vot = parse_single_table(io.BytesIO(r.content))
    tbl = vot.to_table(use_names_over_ids=True)
    print(tbl)
    # first row’s access_url field is the .fits download link
    print(str(tbl["access_url"][0]))
    return str(tbl["access_url"][0])

# 4) Apply to extract fits_url for each
sample["fits_url"] = sample["access_url"].apply(get_fits_url)

# 5) (Optional) Download each FITS
for url in sample["fits_url"]:
    r = requests.get(url)
    r.raise_for_status()
    filename = url.split("/")[-1]
    with open(f"./harps_sample/{filename}", "wb") as f:
        f.write(r.content)
        print('file written')

print("Done. 100 spectra saved to ./harps_sample/")


<Response [200]>
                     ID                      ...
                                             ...
-------------------------------------------- ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
ivo://eso.org/ID?ADP.2014-09-18T12:13:14.680 ...
https://dataportal.eso.org/dataPortal/file/ADP.2014-09-18T12:13:14.680
<Response [200]>
                     ID                      ...
                                             ...
-------------------------------------------- ...
ivo://eso.org/ID?ADP.2014-09-18T12:14:53.390 ...
ivo://eso.org/ID?ADP.2014-09-18T12:14:53.390 ...
ivo://eso.org/ID?ADP.2014-09-18T12:14:53.390 ...
ivo://eso.org/ID?ADP.2014-09-18T12:14:53.390 ...
ivo://eso.org

In [28]:
import os

folder = "harps_sample"
for name in os.listdir(folder):
    if not name.endswith(".fits"):
        src = os.path.join(folder, name)
        dst = os.path.join(folder, name + ".fits")
        os.rename(src, dst)
print("Done renaming", len(os.listdir(folder)), "files.")


Done renaming 100 files.
